In [1]:
import ipywidgets as wdg
from IPython.display import Markdown, HTML, display
from markdown import markdown
from base64 import b64encode

In [140]:
def add_new_button():
    btn = wdg.Button(
        description='Add new',
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
        icon='plus'
    )
    return btn


class HorizontalButtons(object):
    def __init__(self, description):
        btn = add_new_button()
        btn.parent = self
        btn.on_click(self._add_button)
        
        # Create the first input cell (and the `layout` bit is a hack to make the cell the right width)
        first_input = self._new_input()
        first_input.description = description
        first_input.layout = {'height': '100%'}
        self.widget = wdg.HBox(children=[first_input, btn])

    def render(self):
        children = self.widget.children[:-1]
        return [child.value for child in children]
    
    def _new_input(self):
        text = wdg.Text(
            value='',
            placeholder='Enter here',
            disabled=False,
        )
        text.layout.width = "150px"
        return text
        
    def _add_button(self, btn):
        parent = btn.parent
        children = list(parent.widget.children)
        children.insert(-1, self._new_input())
        parent.widget.children = tuple(children)

    def _repr_html_(self):
        display(self.widget)

In [241]:
class BuildPackWidget(object):
    def __init__(self):
        pass
    
    def init_display(self):
        self.gen_downloads_btn = wdg.Button(description="Generate downloads", icon="check")
        self.gen_downloads_btn.on_click(self._generate_download_links)
        self.gen_downloads_btn.layout = {'width': "200px", "margin": "20px 0px 0px 10px"}
        self.widgets_display = wdg.VBox(children=[self.widgets, self.gen_downloads_btn])
        
    def display(self):
        self.init_display()
        display(self.widgets_display)        
        
    def _generate_download_links(self, handler=None):
        # Should be one button per filename
        links = []
        for filename, text in self.render().items():
            payload = b64encode(text.encode('utf-8')).decode()
            download_template = f"<a download='{filename}' href='data:text/plain;charset=utf-8;base64,{payload}'>Download `{filename}`</a>"
            links.append(wdg.HTML(download_template))
        self.widgets_display.children = [self.widgets, self.gen_downloads_btn] + links
    
    def render(self):
        """Return a dictionary of filename: filecontents."""
        pass

    def _repr_html_(self):
        self.display()

# Python
class EnvironmentYml(BuildPackWidget):
    def __init__(self):
        self.name_widget = wdg.Text(description="name", placeholder="my-environment-name")
        self.channel_widget = wdg.Text(description="channels", placeholder="conda-forge")
        self.packages_widget = HorizontalButtons('packages').widget
        self.packages_widget.children[0].placeholder = "numpy=1.16"
        self.widgets = wdg.VBox(children=[self.name_widget, self.channel_widget, self.packages_widget])
        
    def render(self):
        lines = []
        lines.append('name: ' + self.name_widget.value)
        lines.append('channel: ' + self.channel_widget.value)
        lines.append('dependencies:')
        for iline in self.packages_widget.children[:-1]:
            lines.append('  - ' + iline.value)
        return {'environment.yml': '\n'.join(lines)}
        
class Requirements(BuildPackWidget):
    def __init__(self):
        self.packages_widget = HorizontalButtons('packages').widget
        self.packages_widget.children[0].placeholder = "numpy==1.16"
        self.widgets = self.packages_widget
      
    def render(self):
        lines = [line.value for line in self.widgets.children[:-1]]
        return {'requirements.txt ': '\n'.join(lines)}

# R
class RBuildPack(BuildPackWidget):
    def __init__(self):
        self.date_widget = wdg.Text(description="Date", placeholder="YYYY-MM-DD")
        self.install_r_packages_widget = HorizontalButtons('Packages').widget
        self.install_r_packages_widget.children[0].placeholder = "tidyverse"
        self.widgets = wdg.VBox(children=[self.date_widget, self.install_r_packages_widget])
    
    def render(self):
        out = {}
        date = self.date_widget.value
        if len(date.split('-')) != 3:
            raise ValueError("You may not have selected the correct date")
        out['runtime.txt'] = f'r-{date}'
        out['install.R'] = '\n'.join([iline.value for iline in self.install_r_packages_widget.children[:-1]])
        return out

# apt.txt
class Apt(BuildPackWidget):
    def __init__(self):
        self.lines_widget = HorizontalButtons('packages').widget
        self.widgets = self.lines_widget
        
    def render(self):
        lines = [line.value for line in self.widgets.children[:-1]]
        return {'apt.txt ': '\n'.join(lines)}

# runtime.txt
class PostBuild(BuildPackWidget):
    def __init__(self):
        self.lines_widget = HorizontalButtons('commands').widget
        self.widgets = self.lines_widget
        
    def render(self):
        lines = [line.value for line in self.widgets.children[:-1]]
        return {'postBuild ': '\n'.join(lines)}
    
# runtime.txt"
class Start(BuildPackWidget):
    def __init__(self):
        self.lines_widget = HorizontalButtons('commands').widget
        self.widgets = self.lines_widget
        
    def render(self):
        lines = [line.value for line in self.widgets.children[:-1]]
        return {'start ': '\n'.join(lines)}

# Language-specific dependency files

The following sections show how to build configuration files for dependencies in a few
languages.

## Python - `requirements.txt`

A file called `requirements.txt` will use the `pip` package manager to install packages with Python.

In [227]:
req = Requirements()
req.init_display()

req.display()

## python - `environment.yml`

A file called `environment.yml` will use the `conda` package manager to install packages with Anaconda.

In [231]:
env = EnvironmentYml()
env.init_display()

env.display()

## R Dependencies - `runtime.txt` and `install.R`

In order to use an R environment, you'll need two things:

* A **date** of the MRAN repository that defines the state of the packages you wish
  to download. This will go in a file called `runtime.txt`
* A **list of packages** to download. These will be put in a file called `install.R`



In [236]:
rbp = RBuildPack()
rbp.init_display()
rbp.display()

# Extra-scripts and downloads

These configuration files let you perform extra actions after your environment
has been installed.

## Install linux system packages - `apt.txt`

Binder uses a base Ubuntu image as an operating system. If you'd like to install
system packages with `apt-get`, you can list them line-by-line in a file called `apt.txt`.

In [242]:
ap = Apt()
ap.init_display()
ap.display()

## Run arbitrary commands after installing - `postBuild`

A file called `postBuild` will run arbitrary shell commands after installing
the environment with the other configuration files that you've supplied. It's useful
for downloading data, initializing scripts, etc.

In [238]:
pb = PostBuild()
pb.init_display()
pb.display()

## Run a command upon starting the user's session - `start`

Sometimes you'd like to run a command each time the shell starts. For example, if you'd like
to download a small dataset that is often updated. Anything you put in `start` will be run
*whenever a user session starts*.

In [239]:
st = Start()
st.init_display()
st.display()